## Package Installation Check

This section of the code ensures that all necessary packages are installed and loaded for the analysis. The packages we require are `impute`, `tidyverse`, `caret`, and `stringr`. This is particularly important for reproducibility and for sharing the code with others who may not have the required packages installed.

### Implementation Details

- **Persistent Installation Check**: We first check if the `packages_installed` option is already set, which helps us determine if this is the first run of the code in the current R session.
- **Setting Options**: If the packages have not been installed previously (`packages_installed` does not exist), we set this option to `TRUE`. This acts as a flag to prevent re-installation in subsequent runs within the same session.
- **Installation and Loading**:
  - We loop through the list of necessary packages.
  - For each package, we check if it is already installed using the `require()` function. 
  - If not found (`require()` returns `FALSE`), we install the package using `install.packages()` and then load it into the session using `library()`.

### Benefits

- **Efficiency**: This method prevents unnecessary re-installation of packages, saving time and computational resources.
- **User-Friendly**: New users or those running the code in a fresh environment will automatically have the necessary packages installed without manual intervention.
- **Reproducibility**: This approach is essential for ensuring that the code can be run on any machine with minimal setup required by the user.


In [1]:
# Check if the package has been installed in any previous session
if (!exists("packages_installed")) {
  options(packages_installed = TRUE)
  # Install necessary packages
  necessary_packages <- c("tidyverse", "caret", "stringr")
  for (pkg in necessary_packages) {
    if (!require(pkg, character.only = TRUE)) {
      install.packages(pkg)
      library(pkg, character.only = TRUE)
    }
  }
}

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: caret

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


The following object is masked from ‘package:httr’:

    progress




## Custom Imputation Function

The `custom_impute` function is designed to impute missing values in a specified column of a dataframe. It uses a hierarchical matching process based on several criteria. If a sufficient number of similar rows cannot be found, the function removes the row with the missing value.

### Parameters:

- `data`: The dataframe containing the data to be imputed.
- `na_col`: The name of the column in which missing values are to be imputed.

### Process:

1. Iterates through each row in the provided dataframe.
2. Skips rows that do not have a missing value (`NA`) in the specified column.
3. Attempts to find at least 5 similar rows based on the following criteria, in order of priority:
   - `Owner_Type`
   - `Location`
   - `Year`
   - `Make`
4. If similar rows are found, the median of the specified column in these rows is used as the imputed value.
5. If fewer than 5 similar rows are found after removing one criterion at a time, starting with the lowest priority, the row with the missing value is flagged for deletion.

### Deletion of Rows:

- After the imputation process, any row flagged for deletion (due to insufficient similar data) is removed from the dataset.
- A message is printed to the console indicating which rows are being deleted.

### Return Value:

- The function returns the modified dataframe with imputed values and without the rows that had insufficient data for imputation.

### Usage Example:

```r
cleaned_data <- custom_impute(dataframe, "target_column")
## Custom Imputation Function

The `custom_impute` function is designed to impute missing values in a specified column of a dataframe. It uses a hierarchical matching process based on several criteria. If a sufficient number of similar rows cannot be found, the function removes the row with the missing value.

### Parameters:

- `data`: The dataframe containing the data to be imputed.
- `na_col`: The name of the column in which missing values are to be imputed.

### Process:

1. Iterates through each row in the provided dataframe.
2. Skips rows that do not have a missing value (`NA`) in the specified column.
3. Attempts to find at least 5 similar rows based on the following criteria, in order of priority:
   - `Owner_Type`
   - `Location`
   - `Year`
   - `Make`
4. If similar rows are found, the median of the specified column in these rows is used as the imputed value.
5. If fewer than 5 similar rows are found after removing one criterion at a time, starting with the lowest priority, the row with the missing value is flagged for deletion.

### Deletion of Rows:

- After the imputation process, any row flagged for deletion (due to insufficient similar data) is removed from the dataset.
- A message is printed to the console indicating which rows are being deleted.

### Return Value:

- The function returns the modified dataframe with imputed values and without the rows that had insufficient data for imputation.

### Usage Example:

```r
cleaned_data <- custom_impute(dataframe, "target_column")


In [2]:
custom_impute <- function(data, na_col) {
  rows_to_delete <- c() # Keep track of rows to delete
  
  for (i in 1:nrow(data)) {
    if (!is.na(data[i, na_col])) next
    
    all_criteria <- c("Owner_Type", "Location", "Year", "Make")
    criteria_to_use <- all_criteria
    
    found_match <- FALSE
    
    while(length(criteria_to_use) > 0 && !found_match) {
      match_df <- data
      for (crit in criteria_to_use) {
        match_df <- match_df[match_df[[crit]] == data[i, crit], ]
      }
      
      if (nrow(match_df) >= 5) {
        found_match <- TRUE
        imputed_value <- median(match_df[[na_col]], na.rm = TRUE)
        data[i, na_col] <- imputed_value
      } else {
        criteria_to_use <- criteria_to_use[-length(criteria_to_use)] # Remove the last criterion
      }
    }
    
    if (!found_match) {
      rows_to_delete <- c(rows_to_delete, i)
    }
  }
  
  if (length(rows_to_delete) > 0) {
    cat("Deleting rows", paste(rows_to_delete, collapse = ", "), "due to insufficient data for imputation.\n")
    data <- data[-rows_to_delete, ]
  }
  
  return(data)
}


## Reading and Preprocessing Used Car Data

The provided code snippet is responsible for reading a dataset of used car prices and performing initial preprocessing steps.

### Data Reading
- `df`: A variable assigned to the data frame created by reading the CSV file from the specified Kaggle input path.

### Renaming Columns
- The `rename` function within a `dplyr` chain is used to make the dataset columns more descriptive:
  - `Fuel_econ_kmpl`: Renamed from `Mileage` to represent fuel economy in kilometers per liter.
  - `Engine_cc`: Renamed from `Engine` to denote engine capacity in cubic centimeters.
  - `Power_bhp`: Renamed from `Power` to signify the power of the car in brake horsepower.
  - `Price_lakh`: Renamed from `Price` to reflect the price in lakhs (an Indian unit of measurement).

### Cleanup
- The original `df` is removed from memory after renaming the columns to conserve memory and prevent confusion with variable names.

### NA Counting
- `na_count`: A new data frame that stores the count of NA (missing) values across all columns in the `renamed_cols_df` data frame.
- The `summarise_all` function with `~sum(is.na(.))` is used to apply the NA counting operation to each column.

### Output
- The `na_count` data frame will be printed to the console, providing a summary of missing values for each of the renamed columns in the dataset.

This snippet is typically used in the data exploration phase to clean and understand the dataset before proceeding with further analysis or model building.


In [3]:
df = read_csv('/kaggle/input/used-car-prices/train.csv', show_col_types = FALSE)
head(df)

New names:
• `` -> `...1`


...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5,NA,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7,NA,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5,NA,17.74
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5,NA,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8,21 Lakh,17.50


In [4]:
renamed_cols_df <- df %>%
  rename(
    Fuel_econ_kmpl = Mileage,
    Engine_cc = Engine,
    Power_bhp = Power,
    Price_lakh = Price,
  )
na_count = renamed_cols_df %>% 
  summarise_all(~sum(is.na(.)))
na_count


...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Fuel_econ_kmpl,Engine_cc,Power_bhp,Seats,New_Price,Price_lakh
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,2,36,36,38,5032,0


Following the initial data preprocessing, the code continues with additional cleaning steps and feature transformation.

### Dropping Sparse Column
- `new_price_removed_df`: This new data frame is created by removing the `New_Price` column from `renamed_cols_df` since it contains a large number of missing values (only 815 non-missing out of 5847 total instances).

### Parsing Numeric Data
- `parsed_number_df`: In this data frame, certain columns are parsed for numeric content to facilitate mathematical operations and analyses later on.
  - `Fuel_econ_kmpl`: Extracts numeric values from the `Mileage` column, which represents fuel efficiency.
  - `Engine_cc`: Parses numeric values from the `Engine` column, reflecting the engine's capacity.
  - `Power_bhp`: Parses numeric values from the `Power` column to get the power of the car in brake horsepower.
  - `Year`: Converts the `Year` column to an integer type to represent the year properly.

In [5]:
new_price_removed_df <- renamed_cols_df %>% select(-New_Price)
parsed_number_df <- new_price_removed_df %>%
  mutate(
    Fuel_econ_kmpl = parse_number(Fuel_econ_kmpl),
    Engine_cc = parse_number(Engine_cc),
    Power_bhp = parse_number(Power_bhp),
    Year = as.integer(Year, "%Y")
  )
rm(df)

glimpse(parsed_number_df)

Rows: 5,847
Columns: 13
$ ...1              <dbl> 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …
$ Name              <chr> "Hyundai Creta 1.6 CRDi SX Option", "Honda Jazz V", …
$ Location          <chr> "Pune", "Chennai", "Chennai", "Coimbatore", "Jaipur"…
$ Year              <int> 2015, 2011, 2012, 2013, 2013, 2016, 2013, 2012, 2018…
$ Kilometers_Driven <dbl> 41000, 46000, 87000, 40670, 86999, 36000, 64430, 659…
$ Fuel_Type         <chr> "Diesel", "Petrol", "Diesel", "Diesel", "Diesel", "D…
$ Transmission      <chr> "Manual", "Manual", "Manual", "Automatic", "Manual",…
$ Owner_Type        <chr> "First", "First", "First", "Second", "First", "First…
$ Fuel_econ_kmpl    <dbl> 19.67, 13.00, 20.77, 15.20, 23.08, 11.36, 20.54, 22.…
$ Engine_cc         <dbl> 1582, 1199, 1248, 1968, 1461, 2755, 1598, 1248, 1462…
$ Power_bhp         <dbl> 126.20, 88.70, 88.76, 140.80, 63.10, 171.50, 103.60,…
$ Seats             <dbl> 5, 5, 7, 5, 5, 8, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5…
$ Price_lakh    

## Extraction of 'Make' and Calculation of 'Age'

### Feature Engineering from 'Name'
- `make_added_df`: This data frame enhances `parsed_number_df` by deriving new meaningful features.
  - `Make`: Extracts the first word from the `Name` column which represents the brand of the car.
  - `Age`: Calculates the age of the car by subtracting the `Year` from the maximum year in the dataset, taking into account non-missing values (`na.rm = TRUE`).


In [6]:
make_added_df <- parsed_number_df %>%
  mutate(
    Make = str_split(Name, " ", simplify = TRUE)[,1],  # Extract first element (Make)
    Age = max(Year, na.rm = TRUE) - Year  # Calculate Age
  )
rm(renamed_cols_df)
glimpse(make_added_df)

Rows: 5,847
Columns: 15
$ ...1              <dbl> 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …
$ Name              <chr> "Hyundai Creta 1.6 CRDi SX Option", "Honda Jazz V", …
$ Location          <chr> "Pune", "Chennai", "Chennai", "Coimbatore", "Jaipur"…
$ Year              <int> 2015, 2011, 2012, 2013, 2013, 2016, 2013, 2012, 2018…
$ Kilometers_Driven <dbl> 41000, 46000, 87000, 40670, 86999, 36000, 64430, 659…
$ Fuel_Type         <chr> "Diesel", "Petrol", "Diesel", "Diesel", "Diesel", "D…
$ Transmission      <chr> "Manual", "Manual", "Manual", "Automatic", "Manual",…
$ Owner_Type        <chr> "First", "First", "First", "Second", "First", "First…
$ Fuel_econ_kmpl    <dbl> 19.67, 13.00, 20.77, 15.20, 23.08, 11.36, 20.54, 22.…
$ Engine_cc         <dbl> 1582, 1199, 1248, 1968, 1461, 2755, 1598, 1248, 1462…
$ Power_bhp         <dbl> 126.20, 88.70, 88.76, 140.80, 63.10, 171.50, 103.60,…
$ Seats             <dbl> 5, 5, 7, 5, 5, 8, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5…
$ Price_lakh    

## Iterative Imputation Process

This process is designed to iteratively impute missing values across multiple columns within a dataset. It leverages the `custom_impute` function, which selectively imputes data based on a hierarchy of criteria, to ensure that imputed values are reasonable estimates based on similar rows within the dataset.

### Overview:

- The process targets multiple columns for imputation.
- It applies the `custom_impute` function to each targeted column sequentially.
- After imputation, it calculates and displays the number of remaining missing values in the dataset.
- It provides a summary glimpse of the imputed dataframe.

### Targeted Columns for Imputation:

- `Fuel_econ_kmpl`
- `Engine_cc`
- `Power_bhp`
- `Seats`

These columns are specified as they are likely to contain missing values that need to be imputed for further analysis.

### Imputation Process:

1. A list of columns targeted for imputation is defined.
2. The `custom_impute` function is applied to each column in the list.
3. After all specified columns have been processed, the resulting dataframe is stored.


In [7]:
na_cols <- c("Fuel_econ_kmpl", "Engine_cc", "Power_bhp", "Seats")
for (na_col in na_cols) {
    make_added_df = custom_impute(make_added_df, na_col)
}
imputed_df = make_added_df
na_count_post_impute <- imputed_df %>% summarise_all(~sum(is.na(.)))
glimpse(imputed_df)
na_count_post_impute

Deleting rows 710, 724, 1340, 2200, 2267, 2460, 2539, 3171, 3301, 4023, 4100, 4445, 4577, 5122 due to insufficient data for imputation.
Deleting rows 1858 due to insufficient data for imputation.
Rows: 5,832
Columns: 15
$ ...1              <dbl> 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …
$ Name              <chr> "Hyundai Creta 1.6 CRDi SX Option", "Honda Jazz V", …
$ Location          <chr> "Pune", "Chennai", "Chennai", "Coimbatore", "Jaipur"…
$ Year              <int> 2015, 2011, 2012, 2013, 2013, 2016, 2013, 2012, 2018…
$ Kilometers_Driven <dbl> 41000, 46000, 87000, 40670, 86999, 36000, 64430, 659…
$ Fuel_Type         <chr> "Diesel", "Petrol", "Diesel", "Diesel", "Diesel", "D…
$ Transmission      <chr> "Manual", "Manual", "Manual", "Automatic", "Manual",…
$ Owner_Type        <chr> "First", "First", "First", "Second", "First", "First…
$ Fuel_econ_kmpl    <dbl> 19.67, 13.00, 20.77, 15.20, 23.08, 11.36, 20.54, 22.…
$ Engine_cc         <dbl> 1582, 1199, 1248, 1968, 1461, 2755

...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Fuel_econ_kmpl,Engine_cc,Power_bhp,Seats,Price_lakh,Make,Age
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Combining One-Hot Encoded and Selected Features

This section outlines the process of transforming categorical variables into a one-hot encoded matrix and then combining this matrix with the rest of the dataset that contains the remaining selected features.

### Overview:

- Categorical variables are one-hot encoded for use in model building.
- The one-hot encoded dataframe is combined with a selection of other features from the original dataset.
- A structure (`glimpse`) and summary statistics (`summary`) of the combined dataframe are provided for verification and insight.

### Process Details:

1. A model matrix is created using the `model.matrix` function, which converts categorical variables into a one-hot encoded format.
2. The one-hot encoded matrix is coerced into a dataframe.
3. Selected features are extracted from the original imputed dataframe, excluding those that were one-hot encoded or are not required for the model.
4. The one-hot encoded dataframe and the selected features dataframe are combined using `cbind`.
5. The structure and a summary of the combined dataframe are outputted to provide an immediate understanding of the data's composition and characteristics.


In [8]:
model_mat <- model.matrix(~ Fuel_Type + Transmission + Owner_Type +  - 1, data = imputed_df)
one_hot_df <- as.data.frame(model_mat)
selected_df <- imputed_df %>% select(-c(Name, Location, Fuel_Type, Transmission, Owner_Type, Make))

combined_one_hot_df <- cbind(one_hot_df, selected_df)
glimpse(combined_one_hot_df)
summary(combined_one_hot_df)

Rows: 5,832
Columns: 16
$ Fuel_TypeDiesel            <dbl> 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1…
$ Fuel_TypeElectric          <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ Fuel_TypePetrol            <dbl> 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0…
$ TransmissionManual         <dbl> 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1…
$ `Owner_TypeFourth & Above` <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ Owner_TypeSecond           <dbl> 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0…
$ Owner_TypeThird            <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ ...1                       <dbl> 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14,…
$ Year                       <int> 2015, 2011, 2012, 2013, 2013, 2016, 2013, 2…
$ Kilometers_Driven          <dbl> 41000, 46000, 87000, 40670, 86999, 36000, 6…
$ Fuel_econ_kmpl             <dbl> 19.67, 13.00, 20.77, 15.20, 23.08, 11.36, 2…
$ Engine_cc                  <dbl> 1582, 1199, 1248, 1968, 1461, 2755, 1598, 1…
$ Power_bhp     